In [1]:
import pandas as pd
import numpy as np
import time
import os
import collections
from XFLib import *
# class time_est():
#     def __init__(self, total_len):
#         self.t_start = time.time()
#         self.total_len = total_len
#         self.count = 0
#         self.t_ref = time.time()
    
#     def check(self,no_of_check=1,info=""):
#         self.count += no_of_check
#         if time.time() - self.t_ref > 1:
#             t_used = time.time() - self.t_start
#             t_total = t_used * self.total_len / self.count
#             t_remain = t_total - t_used
#             process_bar = "|"
#             for i in range(40):
#                 if (i/40) < (self.count/self.total_len):
#                     process_bar += "█"
#                 else:
#                     process_bar += " "
#             process_bar += "|"
#             print("\r" + (str(info) + "  {:.2f}% ({}/{})\t".format(self.count * 100/self.total_len, self.count,self.total_len)) + str(process_bar).ljust(45) + "Used: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(16) + "ETA: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_remain/3600), int(t_remain/60)%60, t_remain % 60),end="")
#             self.t_ref = time.time()
#         if self.count == self.total_len:
#             t_used = time.time() - self.t_start
#             print("\r" + str(info) + "  Finished in " +"{:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(100))

In [2]:
path = 'i2v_sep/'
file = os.listdir(path)

In [3]:
customer_id_dict = collections.defaultdict()
est = time_est(len(file))
for i in file:
    df = pd.read_csv(path+i, low_memory = False)
    df = df.drop('Unnamed: 0',1)
    df = df[df['day']!='day']
    df['hour'] = [int(x) for x in df['hour']]
    df['minute'] = [int(x) for x in df['minute']]
    df = df.sort_values(['hour','minute'])
    df = df[df['artist_name']!='Various Artists']
    df_np = np.array(df)
    for customer_id, artist_name, _, _, _ in df_np:
        try:
            customer_id_dict[customer_id].append(artist_name)
        except:
            customer_id_dict[customer_id] = []
            customer_id_dict[customer_id].append(artist_name)
    est.check()

Finished in 00:26:59                                                                                            


In [4]:
df['artist_name'].value_counts()

Ed Sheeran                                                                     983568
David Guetta                                                                   272640
Coldplay                                                                       245570
Clean Bandit                                                                   230972
Paolo Nutini                                                                   130822
Vance Joy                                                                      112625
Red Hot Chili Peppers                                                          103934
Nico & Vinz                                                                    102628
Fleetwood Mac                                                                   98474
Bruno Mars                                                                      90198
Linkin Park                                                                     87057
LILLYWOOD & Robin Schulz                              

In [5]:
np.save('customer_id_dict.npy', customer_id_dict)

In [6]:
artist_info = list(customer_id_dict.values())

In [7]:
len(artist_info)

4425980

In [8]:
import gensim

In [9]:
model = gensim.models.Word2Vec(artist_info, size=100, window=5, min_count=5, workers=4)

In [10]:
model.save('i2v/i2v')

In [102]:
model.most_similar('Linkin Park')

[('Transformers: Revenge Of The Fallen Soundtrack', 0.6466086506843567),
 ('LINKIN PARK', 0.6428512930870056),
 ('Jay-Z', 0.6170141100883484),
 ('Busta Rhymes feat. Linkin Park', 0.5231870412826538),
 ('LINKIN PARK x STEVE AOKI', 0.4748544692993164),
 ('P.O.D.', 0.44549843668937683),
 ('Trapt', 0.44295287132263184),
 ('Egypt Central', 0.42924609780311584),
 ('Transformers - The Album', 0.4272550940513611),
 ('Dead By Sunrise', 0.41367942094802856)]